# Module 5: Zero-Noise Extrapolation (ZNE) – The Physics Baseline

**Status:** Scientifically Validated (Polynomial Fit) ✅

Before we jump to AI, we must master the physics-based method: **Zero-Noise Extrapolation (ZNE)**. It is the industry standard for error mitigation.

## 5.1 The Concept: "Turning Up The Static"

Imagine you are listening to a radio with static noise level $\lambda=1$. You cannot turn the static *down* (because it's hardware noise), but you **can** turn it *up*.

If you measure the signal quality at:
*   Noise $\lambda=1$ (Baseline)
*   Noise $\lambda=2$ (Double Static)
*   Noise $\lambda=3$ (Triple Static)

You can plot these points and draw a curve backwards to $\lambda=0$ (Zero Noise). This is **Extrapolation**.

## 5.2 The Math: Richardson Extrapolation

The expectation value $E(\lambda)$ can be approximated as a Taylor series:
$$ E(\lambda) = E(0) + a_1 \lambda + a_2 \lambda^2 + \dots $$

**Crucial Insight:** For shallow circuits, the noise is linear ($a_2 \approx 0$). For deep circuits, the noise is exponential (curved), so a linear fit fails. We use a **Polynomial Fit (Degree 2)** to capture this curvature.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import utils  # Using shared noise model

# --- 1. Robust ZNE Logic ---
def zne_extrapolate_poly(noisy_values, noise_scales=[1, 2, 3], degree=2):
    """
    Fits a polynomial (default quadratic) to the noisy points.
    Returns prediction at scale=0.
    
    Why Degree=2? 
    Because decay is typically exponential: y = A * exp(-x).
    A quadratic (parabola) approximates the start of an exponential curve much better than a line.
    """
    # Fit y = ax^2 + bx + c
    coeffs = np.polyfit(noise_scales, noisy_values, degree)
    
    # Evaluate at x=0 (which is just the last coefficient 'c')
    zero_noise_val = np.polyval(coeffs, 0)
    return zero_noise_val, coeffs

# --- 2. Run the Experiment ---
# We construct a deep circuit where linear ZNE would likely fail
qc = QuantumCircuit(2)
for _ in range(5): # Depth 5
    qc.h(0)
    qc.cx(0, 1)
qc.measure_all()
ideal_val = 1.0 # Should be |00> + |11> (Einstein-Podolsky-Rosen pair)

scales = [1.0, 2.0, 3.0, 4.0] # We take 4 points to comfortably fit a degree-2 curve
results = []

print("Running ZNE Experiments (Pulse Stretching Simulation)...")
for s in scales:
    # utils.build_custom_noise_model scales T1/T2 times inversely
    # effectively simulating a 'slower' pulse that accumulates more noise
    nm = utils.build_custom_noise_model(noise_scale=s)
    sim = AerSimulator(noise_model=nm)
    
    # Run
    job = sim.run(transpile(qc, sim), shots=4000) 
    counts = job.result().get_counts()
    
    # Calc <ZZ>
    shots = sum(counts.values())
    p_even = (counts.get('00', 0) + counts.get('11', 0)) / shots
    p_odd = (counts.get('01', 0) + counts.get('10', 0)) / shots
    exp_val = p_even - p_odd
    
    results.append(exp_val)
    print(f"  Scale {s}x -> Expectation: {exp_val:.3f}")

# --- 3. Compare Linear vs Poly ---
# This demonstrates WHY math matters. Linear ZNE might overshoot.
mit_poly, coeffs = zne_extrapolate_poly(results, scales, degree=2)
mit_linear, coeffs_lin = zne_extrapolate_poly(results, scales, degree=1)

print(f"\n--- RESULTS ---")
print(f"  Noisy Baseline (1x): {results[0]:.3f}")
print(f"  Linear ZNE:          {mit_linear:.3f}")
print(f"  Polynomial ZNE:      {mit_poly:.3f} (More accurate for curves)")
print(f"  Ideal Target:        {ideal_val:.3f}")

# --- 4. Plot ---
plt.figure(figsize=(8, 5))
plt.plot(scales, results, 'ro', label='Measured Points')

# Plot fits
x_line = np.linspace(0, 4.5, 50)
plt.plot(x_line, np.polyval(coeffs, x_line), 'g-', label='Quadratic Fit (Poly ZNE)')
plt.plot(x_line, np.polyval(coeffs_lin, x_line), 'b--', label='Linear Fit (Simple ZNE)')

# Plot prediction
plt.plot(0, mit_poly, 'g*', markersize=15, label='Poly Prediction')
plt.plot(0, mit_linear, 'b*', markersize=10, alpha=0.5, label='Linear Prediction')

plt.axhline(ideal_val, color='gray', linestyle=':', label='Ideal')
plt.xlabel('Noise Scale factor ($\lambda$)')
plt.ylabel('Expectation Value <ZZ>')
plt.title('Why Math Matters: Linear vs Polynomial ZNE')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5.3 Conclusion
Notice how the **Quadratic Fit** (Green) often lands closer to the Ideal Value than the Linear Fit (Blue), especially if the points are curving downwards.

However, ZNE alone is rarely enough. It mitigates the "average" noise, but it cannot fix coherent shifts or memory effects. That is where we need AI (Module 6).